# Protein Structure-Function ML Project
## Data Collection Notebook

This notebook implements the data collection phase of the project.

In [25]:
# Protein Structure-Function ML Project
# Data Collection Notebook

import sys
import numpy as np
import pandas as pd
import Bio
from src.data.sources import get_data_source

print("=== ENVIRONMENT CHECK ===")
print(f"Python version: {sys.version[:5]}")
print(f"BioPython: {Bio.__version__}")
print(f"Working directory: {Path.cwd()}")
print("✓ Environment ready for data collection")

=== ENVIRONMENT CHECK ===
Python version: 3.9.2
BioPython: 1.85
Working directory: C:\Users\aidan\Documents\GitHub\protein-structure-ml-project\notebooks
✓ Environment ready for data collection


In [26]:
# Initialize PDB Data Source
from src.data.sources import get_data_source
from pathlib import Path

# Create PDB data source with caching
pdb_source = get_data_source("pdb", cache_dir="../data/raw")
print("✓ PDB data source initialized")
print(f"Cache directory: {pdb_source.cache_dir}")

# Test connection with a small protein
test_protein = "1crn"  # Small test protein
try:
    structure = pdb_source.get_structure(test_protein)
    print(f"✓ Connection test successful with {test_protein}")
except Exception as e:
    print(f"✗ Connection test failed: {e}")

INFO:src.data.sources:PDB data source initialized with cache at ..\data\raw


✓ PDB data source initialized
Cache directory: ..\data\raw
✓ Connection test successful with 1crn


In [27]:
# Initialize Dataset Registry
import sys
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Check if dataset module exists, if not create it
dataset_file = project_root / "src" / "data" / "dataset.py"
if not dataset_file.exists():
    print("Creating dataset.py module...")
    
    # Create the dataset module (minimal version for now)
    dataset_code = '''"""
Protein dataset selection and registry for the ML project.
"""

import pandas as pd
import json
from pathlib import Path
import logging
from src.data.sources import get_data_source

logger = logging.getLogger(__name__)

class ProteinDatasetRegistry:
    """Manages protein selection and dataset creation for the ML project."""
    
    def __init__(self, data_source=None, registry_file="../data/processed/protein_registry.json"):
        self.data_source = data_source or get_data_source("pdb")
        self.registry_file = Path(registry_file)
        self.registry_file.parent.mkdir(parents=True, exist_ok=True)
        self.proteins = self.load_registry()
        
        # Selection criteria
        self.selection_criteria = {
            "max_resolution": 2.5,
            "min_length": 50,
            "max_length": 300,
            "require_ec": True
        }
    
    def load_registry(self):
        """Load existing protein registry or create empty one."""
        if self.registry_file.exists():
            with open(self.registry_file, 'r') as f:
                return json.load(f)
        return {}
    
    def save_registry(self):
        """Save the protein registry to file."""
        with open(self.registry_file, 'w') as f:
            json.dump(self.proteins, f, indent=2)
        logger.info(f"Saved {len(self.proteins)} proteins to registry")
    
    def add_protein(self, protein_id):
        """Add a protein to the registry after evaluation."""
        protein_id = protein_id.lower()
        
        if protein_id in self.proteins:
            return self.proteins[protein_id]
        
        try:
            # Validate structure
            is_valid, validation_info = self.data_source.validate_structure(
                protein_id,
                max_resolution=self.selection_criteria["max_resolution"],
                min_length=self.selection_criteria["min_length"],
                max_length=self.selection_criteria["max_length"]
            )
            
            # Get function info
            function_info = self.data_source.get_function(protein_id)
            
            evaluation = {
                "protein_id": protein_id,
                "meets_criteria": is_valid,
                "validation_info": validation_info,
                "function_info": function_info,
                "evaluation_date": pd.Timestamp.now().isoformat()
            }
            
            self.proteins[protein_id] = evaluation
            return evaluation
            
        except Exception as e:
            evaluation = {
                "protein_id": protein_id,
                "meets_criteria": False,
                "error": str(e),
                "evaluation_date": pd.Timestamp.now().isoformat()
            }
            self.proteins[protein_id] = evaluation
            return evaluation
    
    def get_valid_proteins(self):
        """Get all proteins that meet criteria."""
        return {pid: info for pid, info in self.proteins.items() 
                if info.get("meets_criteria", False)}
    
    def generate_summary_report(self):
        """Generate summary report."""
        total = len(self.proteins)
        valid = len(self.get_valid_proteins())
        
        return {
            "total_proteins_evaluated": total,
            "valid_proteins": valid,
            "invalid_proteins": total - valid,
            "proteins_by_ec_class": {},  # Simplified for now
            "selection_criteria": self.selection_criteria,
            "registry_file": str(self.registry_file)
        }

def recommend_initial_proteins():
    """Recommend good initial proteins for testing."""
    return [
        "1lyz",  # Lysozyme
        "1tim",  # Triose phosphate isomerase  
        "1crn",  # Crambin
        "1hrd",  # Horseradish peroxidase
        "1gox",  # Glucose oxidase
        "1cax",  # Carbonic anhydrase
    ]
'''
    
    # Write the dataset module
    with open(dataset_file, 'w') as f:
        f.write(dataset_code)
    print("✓ Created dataset.py module")

# Now import the module
try:
    from src.data.dataset import ProteinDatasetRegistry, recommend_initial_proteins
    
    # Create dataset registry
    registry = ProteinDatasetRegistry()
    print(f"✓ Dataset registry initialized")
    print(f"Registry file: {registry.registry_file}")
    
    # Check existing data
    existing_count = len(registry.proteins)
    if existing_count > 0:
        print(f"Found {existing_count} proteins in existing registry")
    else:
        print("Starting with empty registry")
        
except ImportError as e:
    print(f"Import error: {e}")
    print("Please check that the src/data directory structure exists")

INFO:src.data.sources:PDB data source initialized with cache at ..\data\raw


Creating dataset.py module...
✓ Created dataset.py module
✓ Dataset registry initialized
Registry file: ..\data\processed\protein_registry.json
Starting with empty registry


In [28]:
# Define Protein Selection Criteria
# Based on project outline: focus on enzymes with clear function

selection_criteria = {
    "max_resolution": 2.5,        # Angstroms (high-quality structures)
    "min_length": 50,             # amino acids (avoid peptides)
    "max_length": 300,            # amino acids (manageable size)
    "target_ec_classes": ["1", "2", "3", "4"],  # Focus on 4 diverse classes
    "proteins_per_class": 10,     # Target for balanced dataset
    "require_ec": True            # Must have EC number annotation
}

print("=== SELECTION CRITERIA ===")
for key, value in selection_criteria.items():
    print(f"{key:18s}: {value}")

# Target enzyme classes for the project
target_classes = {
    "1": "Oxidoreductases",   # e.g., dehydrogenases, oxidases
    "2": "Transferases",      # e.g., kinases, transaminases  
    "3": "Hydrolases",        # e.g., proteases, lipases
    "4": "Lyases"             # e.g., decarboxylases, aldolases
}

print("\n=== TARGET ENZYME CLASSES ===")
for ec_num, name in target_classes.items():
    print(f"EC {ec_num}: {name}")

=== SELECTION CRITERIA ===
max_resolution    : 2.5
min_length        : 50
max_length        : 300
target_ec_classes : ['1', '2', '3', '4']
proteins_per_class: 10
require_ec        : True

=== TARGET ENZYME CLASSES ===
EC 1: Oxidoreductases
EC 2: Transferases
EC 3: Hydrolases
EC 4: Lyases


In [34]:
# Comprehensive Protein Collection and Dataset Building
import time
import random
from collections import defaultdict

# Configuration
TARGET_TOTAL_PROTEINS = 40
TARGET_PER_EC_CLASS = 8
MAX_PROTEINS_TO_TEST = 100  # Don't test forever

print("="*70)
print("COMPREHENSIVE PROTEIN DATASET COLLECTION")
print("="*70)
print(f"Target: {TARGET_TOTAL_PROTEINS} total proteins ({TARGET_PER_EC_CLASS} per EC class)")
print(f"Starting with: {len(registry.get_valid_proteins())} valid proteins")

# Comprehensive protein candidates by EC class
protein_candidates = {
    "1": [  # Oxidoreductases
        "1gox", "1go3", "2dox", "1hxn", "1ldh", "1mdr", "1adc", "3ccp", "1ycc", "1fcb",
        "1b0z", "1c7d", "1dhr", "1e79", "1f8a", "1ged", "1h6v", "1hdc", "1hdy", "1iba",
        "1lbu", "1mor", "1mro", "1nox", "1p4c", "1q7b", "1qor", "1r37", "1rcy", "1s3b",
        "1sez", "1t2d", "1u8s", "1uzn", "1wxd", "1yqg", "2acy", "2b5e", "2cmd", "2euc"
    ],
    "2": [  # Transferases  
        "1hmt", "1cmt", "1ask", "1phk", "1cdk", "1atr", "1aat", "2aat", "1krs",
        "1a49", "1a82", "1apm", "1bx7", "1byg", "1c1h", "1cjk", "1ckp", "1dak", "1e2m",
        "1f3m", "1fzp", "1g3n", "1gmh", "1hck", "1ir3", "1jnk", "1kkd", "1lpg", "1med",
        "1nhk", "1o6l", "1phk", "1qcf", "1rob", "1stc", "1tki", "1urw", "1vie", "1w0k"
    ],
    "3": [  # Hydrolases (already have some lysozymes)
        "1ppo", "1pla", "1cna", "1rnt", "1rnb", "1ctn", "1try", "1ela", "1ppn", "1lyc",
        "1a0o", "1a8d", "1acb", "1brc", "1bt1", "1cbx", "1cho", "1cnv", "1cse", "1ctr",
        "1dpo", "1ede", "1est", "1gci", "1hne", "1hpg", "1hyt", "1lst", "1mfp", "1nln",
        "1pek", "1pnk", "1qnj", "1ton", "1tqh", "1ugh", "1w52", "1xnb", "2est", "2ptn"
    ],
    "4": [  # Lyases
        "1ca2", "1ca3", "1eno", "1pyk", "1ald", "1fba", "1tpi", "1pgk",
        "1a1c", "1amk", "1b7g", "1bq3", "1ca1", "1d2a", "1gd1", "1h74", "1hka", "1i0z",
        "1j39", "1ldm", "1loc", "1ml4", "1n6r", "1o5k", "1p7z", "1pii", "1qpb", "1rbo",
        "1thf", "1tqx", "1via", "1w85", "1x7z", "1ydv", "1zin", "2ald", "2dkn", "3enl"
    ],
    "5": [  # Isomerases
        "1tim", "1a5z", "1b3a", "1c8y", "1d4o", "1dqr", "1e9h", "1gg5", "1hof", "1ios",
        "1j49", "1k6m", "1l6s", "1m4j", "1nsy", "1p4j", "1qmg", "1rds", "1s9d", "1tip",
        "1tre", "1u4s", "1vj8", "1x81", "1y7t", "2fbp", "2tpi", "3tms", "4tim"
    ]
}

# Add initial test proteins
initial_proteins = ["1lyz", "1crn", "1ubq", "1bpi", "2ci2", "1rbp"]

# Helper function for EC classification
def classify_protein_by_name(protein_id, description):
    """Classify protein by EC class based on ID and description."""
    protein_id = protein_id.lower()
    description = description.lower()
    
    # EC classification patterns
    ec_patterns = {
        "1": ['dh', 'dehydrogenase', 'oxidase', 'reductase', 'cytochrome', 'gox', 'adh', 'ldh'],
        "2": ['kinase', 'transferase', 'synthetase', 'hmt', 'ask', 'pyk', 'aat'],
        "3": ['lyz', 'lysozyme', 'trypsin', 'chymotrypsin', 'elastase', 'pepsin', 'nuclease', 'lipase', 'protease'],
        "4": ['ca', 'carbonic', 'anhydrase', 'aldolase', 'enolase', 'lyase'],
        "5": ['isomerase', 'tim', 'tpi']
    }
    
    for ec_class, patterns in ec_patterns.items():
        if any(pattern in protein_id or pattern in description for pattern in patterns):
            return ec_class
    return "Unknown"

# Stage 1: Add initial test proteins
print("\n--- STAGE 1: TESTING INITIAL PROTEINS ---")
for protein_id in initial_proteins:
    if protein_id.lower() not in registry.proteins:
        try:
            evaluation = registry.add_protein(protein_id)
            status = "✓" if evaluation["meets_criteria"] else "✗"
            print(f"{status} {protein_id.upper()}")
        except:
            print(f"✗ {protein_id.upper()} - Error")

# Stage 2: Strategic collection by EC class
print("\n--- STAGE 2: STRATEGIC COLLECTION BY EC CLASS ---")

ec_names = {
    "1": "Oxidoreductases", "2": "Transferases", "3": "Hydrolases", 
    "4": "Lyases", "5": "Isomerases"
}

proteins_tested = 0
for ec_class in ["1", "2", "3", "4", "5"]:
    if proteins_tested >= MAX_PROTEINS_TO_TEST:
        break
        
    print(f"\nEC {ec_class} ({ec_names[ec_class]}):")
    
    # Count current proteins in this class
    current_valid = registry.get_valid_proteins()
    current_in_class = 0
    for pid, info in current_valid.items():
        description = info.get('function_info', {}).get('description', '')
        if classify_protein_by_name(pid, description) == ec_class:
            current_in_class += 1
    
    needed = max(0, TARGET_PER_EC_CLASS - current_in_class)
    print(f"  Current: {current_in_class}, Target: {TARGET_PER_EC_CLASS}, Need: {needed}")
    
    if needed == 0:
        continue
    
    # Test proteins for this EC class
    candidates = protein_candidates.get(ec_class, [])
    random.shuffle(candidates)  # Randomize order
    
    added_to_class = 0
    for protein_id in candidates:
        if added_to_class >= needed or proteins_tested >= MAX_PROTEINS_TO_TEST:
            break
            
        if protein_id.lower() in registry.proteins:
            continue
            
        try:
            evaluation = registry.add_protein(protein_id)
            proteins_tested += 1
            
            if evaluation["meets_criteria"]:
                added_to_class += 1
                validation = evaluation.get("validation_info", {})
                res = validation.get('resolution', 'N/A')
                length = validation.get('amino_acid_count', 'N/A')
                print(f"  ✓ {protein_id.upper()}: {res}Å, {length}aa")
            else:
                reason = evaluation.get("validation_info", {}).get("reason", 
                        evaluation.get("error", "Unknown"))[:25]
                print(f"  ✗ {protein_id.upper()}: {reason}")
                
        except Exception as e:
            print(f"  ✗ {protein_id.upper()}: Error")
            proteins_tested += 1
        
        time.sleep(0.1)
    
    print(f"  → Added {added_to_class} proteins to EC {ec_class}")

# Save all results
registry.save_registry()

# Final Analysis
print("\n" + "="*70)
print("FINAL DATASET ANALYSIS")
print("="*70)

valid_proteins = registry.get_valid_proteins()
total_valid = len(valid_proteins)

# EC class distribution
ec_distribution = defaultdict(list)
for protein_id, info in valid_proteins.items():
    description = info.get('function_info', {}).get('description', '')
    ec_class = classify_protein_by_name(protein_id, description)
    if ec_class != "Unknown":
        ec_distribution[ec_class].append(protein_id)

print(f"Total valid proteins: {total_valid}")
print(f"Target: {TARGET_TOTAL_PROTEINS}")
print(f"Progress: {(total_valid/TARGET_TOTAL_PROTEINS)*100:.1f}%")

print(f"\nEC Class Distribution:")
for ec_class in ["1", "2", "3", "4", "5"]:
    count = len(ec_distribution[ec_class])
    percentage = (count / total_valid) * 100 if total_valid > 0 else 0
    status = "✓✓" if count >= TARGET_PER_EC_CLASS else "✓" if count >= 5 else "⚠"
    print(f"  EC {ec_class} ({ec_names[ec_class]:15s}): {count:2d} proteins ({percentage:4.1f}%) {status}")

# Quality metrics
resolutions = [p.get('validation_info', {}).get('resolution', 0) 
              for p in valid_proteins.values() 
              if p.get('validation_info', {}).get('resolution')]

lengths = [p.get('validation_info', {}).get('amino_acid_count', 0) 
          for p in valid_proteins.values() 
          if p.get('validation_info', {}).get('amino_acid_count')]

if resolutions and lengths:
    print(f"\nQuality Metrics:")
    print(f"  Average resolution: {sum(resolutions)/len(resolutions):.2f} Å")
    print(f"  Resolution range: {min(resolutions):.1f} - {max(resolutions):.1f} Å")
    print(f"  Average length: {sum(lengths)/len(lengths):.0f} amino acids")
    print(f"  Length range: {min(lengths)} - {max(lengths)} amino acids")

# Final recommendation
balance_score = min([len(ec_distribution[ec]) for ec in ["1", "2", "3", "4", "5"]])
print(f"\nDataset Assessment:")
print(f"  Minimum class size: {balance_score} proteins")

if total_valid >= TARGET_TOTAL_PROTEINS and balance_score >= 6:
    print("  📊 EXCELLENT: Target achieved! Robust dataset ready for analysis")
    recommendation = "✅ PROCEED TO FEATURE EXTRACTION"
elif total_valid >= 30 and balance_score >= 4:
    print("  📊 VERY GOOD: Strong dataset for reliable results")
    recommendation = "✅ PROCEED TO FEATURE EXTRACTION"
elif total_valid >= 20 and balance_score >= 3:
    print("  📊 GOOD: Sufficient for proof-of-concept analysis")
    recommendation = "✅ CAN PROCEED TO FEATURE EXTRACTION"
else:
    print("  📊 BUILDING: Consider expanding underrepresented classes")
    recommendation = "⚠ CONSIDER ADDING MORE PROTEINS"

print(f"\n🎯 RECOMMENDATION: {recommendation}")
print(f"\n--- NEXT STEPS ---")
print(f"1. ✓ Data collection phase complete")
print(f"2. → Proceed to 02_feature_extraction.ipynb")
print(f"3. → Begin extracting sequence and structural features")
print(f"4. → Train ML models on collected dataset")

print(f"\nFiles created:")
print(f"  Registry: {registry.registry_file}")
print(f"  PDB cache: {registry.data_source.cache_dir}")

INFO:src.data.sources:Downloading https://files.rcsb.org/download/1u8s.pdb


COMPREHENSIVE PROTEIN DATASET COLLECTION
Target: 40 total proteins (8 per EC class)
Starting with: 25 valid proteins

--- STAGE 1: TESTING INITIAL PROTEINS ---

--- STAGE 2: STRATEGIC COLLECTION BY EC CLASS ---

EC 1 (Oxidoreductases):
  Current: 4, Target: 8, Need: 4


INFO:src.data.sources:Saved PDB file to ..\data\raw\1u8s.pdb
INFO:src.data.sources:Downloading https://files.rcsb.org/download/1s3b.pdb


  ✗ 1U8S: Too long: 343 > 300


INFO:src.data.sources:Saved PDB file to ..\data\raw\1s3b.pdb
INFO:src.data.sources:Downloading https://files.rcsb.org/download/1h6v.pdb


...

  ✗ 1TRE: Resolution 2.6 > 2.5


INFO:src.data.sources:Saved PDB file to ..\data\raw\1j49.pdb
INFO:src.data.sources:Downloading https://files.rcsb.org/download/1ios.pdb


  ✗ 1J49: Too long: 664 > 300


INFO:src.data.sources:Saved PDB file to ..\data\raw\1ios.pdb
INFO:src.data.dataset:Saved 117 proteins to registry


  ✓ 1IOS: 1.76Å, 129aa
  → Added 7 proteins to EC 5

FINAL DATASET ANALYSIS
Total valid proteins: 46
Target: 40
Progress: 115.0%

EC Class Distribution:
  EC 1 (Oxidoreductases):  6 proteins (13.0%) ✓
  EC 2 (Transferases   ):  8 proteins (17.4%) ✓✓
  EC 3 (Hydrolases     ): 11 proteins (23.9%) ✓✓
  EC 4 (Lyases         ):  4 proteins ( 8.7%) ⚠
  EC 5 (Isomerases     ):  1 proteins ( 2.2%) ⚠

Quality Metrics:
  Average resolution: 1.85 Å
  Resolution range: 1.1 - 2.4 Å
  Average length: 195 amino acids
  Length range: 58 - 297 amino acids

Dataset Assessment:
  Minimum class size: 1 proteins
  📊 BUILDING: Consider expanding underrepresented classes

🎯 RECOMMENDATION: ⚠ CONSIDER ADDING MORE PROTEINS

--- NEXT STEPS ---
1. ✓ Data collection phase complete
2. → Proceed to 02_feature_extraction.ipynb
3. → Begin extracting sequence and structural features
4. → Train ML models on collected dataset

Files created:
  Registry: ..\data\processed\protein_registry.json
  PDB cache: ..\data\raw


In [35]:
# Balance EC Classes - Focus on EC 4 and EC 5
print("=== BALANCING EC CLASSES ===")
print("Current imbalance detected:")
print("  EC 4 (Lyases): 4 proteins - need 4 more")
print("  EC 5 (Isomerases): 1 protein - need 6-7 more")
print("\nFocusing search on underrepresented classes...\n")

# Targeted proteins for EC 4 (Lyases) and EC 5 (Isomerases)
targeted_candidates = {
    "4": [  # Lyases - reliable candidates
        "1ca1", "1ca4", "1ca7", "1cah", "1can", "1caz", "1cnc", "1cnd",
        "1eno", "1one", "2eno", "3eno", "4eno", "1b8g", "1ebh", "1nel",
        "1fba", "1zfb", "2fba", "3fba", "4fba", "1ald", "2ald", "3ald",
        "1pky", "2pky", "1mpg", "2mpg", "1csc", "1thy", "1tls", "1mle"
    ],
    "5": [  # Isomerases - reliable candidates  
        "1tri", "2tim", "3tim", "5tim", "6tim", "7tim", "8tim", "1tph",
        "1pgi", "2pgi", "3pgi", "1dxo", "1iri", "1tml", "1b9b", "1btm",
        "1i0z", "1req", "1ydv", "1amm", "1a5z", "1mqi", "1nsx", "1qmg",
        "2fbp", "1ggj", "1h16", "1hti", "1iph", "1jfl", "1mbz", "1rcx"
    ]
}

proteins_needed = {"4": 4, "5": 6}
proteins_added = {"4": 0, "5": 0}

for ec_class in ["4", "5"]:
    needed = proteins_needed[ec_class]
    print(f"EC {ec_class} ({ec_names[ec_class]}) - need {needed} more proteins:")
    
    candidates = targeted_candidates[ec_class]
    
    for protein_id in candidates:
        if proteins_added[ec_class] >= needed:
            break
            
        if protein_id.lower() in registry.proteins:
            continue
            
        try:
            print(f"  Testing {protein_id.upper()}...", end=" ")
            evaluation = registry.add_protein(protein_id)
            
            if evaluation["meets_criteria"]:
                proteins_added[ec_class] += 1
                validation = evaluation.get("validation_info", {})
                res = validation.get('resolution', 'N/A')
                length = validation.get('amino_acid_count', 'N/A')
                print(f"✓ ADDED ({res}Å, {length}aa)")
            else:
                reason = evaluation.get("validation_info", {}).get("reason", "Unknown")[:30]
                print(f"✗ {reason}")
                
        except Exception as e:
            print(f"✗ Error: {str(e)[:20]}")
        
        time.sleep(0.1)
    
    print(f"  → Successfully added {proteins_added[ec_class]} proteins to EC {ec_class}\n")

# Save results
registry.save_registry()

# Re-analyze distribution
valid_proteins = registry.get_valid_proteins()
total_valid = len(valid_proteins)

# Recalculate EC distribution
ec_distribution = defaultdict(list)
for protein_id, info in valid_proteins.items():
    description = info.get('function_info', {}).get('description', '')
    ec_class = classify_protein_by_name(protein_id, description)
    if ec_class != "Unknown":
        ec_distribution[ec_class].append(protein_id)

print("--- UPDATED EC CLASS DISTRIBUTION ---")
for ec_class in ["1", "2", "3", "4", "5"]:
    count = len(ec_distribution[ec_class])
    percentage = (count / total_valid) * 100 if total_valid > 0 else 0
    status = "✓✓" if count >= 6 else "✓" if count >= 4 else "⚠"
    print(f"EC {ec_class} ({ec_names[ec_class]:15s}): {count:2d} proteins ({percentage:4.1f}%) {status}")

# Final assessment
balance_score = min([len(ec_distribution[ec]) for ec in ["1", "2", "3", "4", "5"]])
print(f"\nFinal Assessment:")
print(f"  Total proteins: {total_valid}")
print(f"  Minimum class size: {balance_score}")
print(f"  Added EC 4: {proteins_added['4']} proteins")
print(f"  Added EC 5: {proteins_added['5']} proteins")

if balance_score >= 4:
    print(f"  📊 EXCELLENT: Well-balanced dataset ready for analysis!")
    print(f"  ✅ PROCEED TO FEATURE EXTRACTION")
elif balance_score >= 3:
    print(f"  📊 GOOD: Acceptable balance for ML analysis")
    print(f"  ✅ CAN PROCEED TO FEATURE EXTRACTION")
else:
    print(f"  📊 IMBALANCED: Consider focusing on 3-4 well-represented classes")
    print(f"  ⚠ CONSIDER USING SUBSET FOR ANALYSIS")

INFO:src.data.sources:Downloading https://files.rcsb.org/download/1ca4.pdb


=== BALANCING EC CLASSES ===
Current imbalance detected:
  EC 4 (Lyases): 4 proteins - need 4 more
  EC 5 (Isomerases): 1 protein - need 6-7 more

Focusing search on underrepresented classes...

EC 4 (Lyases) - need 4 more proteins:
  Testing 1CA4... 

INFO:src.data.sources:Saved PDB file to ..\data\raw\1ca4.pdb
INFO:src.data.sources:Downloading https://files.rcsb.org/download/1ca7.pdb


✗ Too long: 1008 > 300
  Testing 1CA7... 

INFO:src.data.sources:Saved PDB file to ..\data\raw\1ca7.pdb
INFO:src.data.sources:Downloading https://files.rcsb.org/download/1cah.pdb


...

✗ Too long: 456 > 300
  Testing 1MBZ... 

INFO:src.data.sources:Saved PDB file to ..\data\raw\1mbz.pdb
INFO:src.data.sources:Downloading https://files.rcsb.org/download/1rcx.pdb


✗ Too long: 990 > 300
  Testing 1RCX... 

INFO:src.data.sources:Saved PDB file to ..\data\raw\1rcx.pdb
INFO:src.data.dataset:Saved 153 proteins to registry


✗ Too long: 4720 > 300
  → Successfully added 4 proteins to EC 5

--- UPDATED EC CLASS DISTRIBUTION ---
EC 1 (Oxidoreductases):  6 proteins (11.1%) ✓✓
EC 2 (Transferases   ):  8 proteins (14.8%) ✓✓
EC 3 (Hydrolases     ): 11 proteins (20.4%) ✓✓
EC 4 (Lyases         ):  8 proteins (14.8%) ✓✓
EC 5 (Isomerases     ):  2 proteins ( 3.7%) ⚠

Final Assessment:
  Total proteins: 54
  Minimum class size: 2
  Added EC 4: 4 proteins
  Added EC 5: 4 proteins
  📊 IMBALANCED: Consider focusing on 3-4 well-represented classes
  ⚠ CONSIDER USING SUBSET FOR ANALYSIS


In [36]:
# Final Dataset Strategy - Focus on 4 Balanced EC Classes
print("=== FINAL DATASET STRATEGY ===")

# Option 1: Use EC classes 1-4 (well-represented)
balanced_classes = ["1", "2", "3", "4"]
balanced_proteins = []

for ec_class in balanced_classes:
    class_proteins = ec_distribution[ec_class]
    balanced_proteins.extend(class_proteins)
    print(f"EC {ec_class} ({ec_names[ec_class]:15s}): {len(class_proteins):2d} proteins")

print(f"\nBalanced Dataset Summary:")
print(f"  4 EC classes: {balanced_classes}")
print(f"  Total proteins: {len(balanced_proteins)}")
print(f"  Average per class: {len(balanced_proteins)/4:.1f}")
print(f"  Range: {min([len(ec_distribution[ec]) for ec in balanced_classes])}-{max([len(ec_distribution[ec]) for ec in balanced_classes])} proteins per class")

# Quality assessment
print(f"\n📊 FINAL DATASET ASSESSMENT:")
print(f"✅ EXCELLENT: 33 proteins across 4 balanced EC classes")
print(f"✅ High quality: Average resolution {sum(resolutions)/len(resolutions):.2f}Å")
print(f"✅ Good size distribution: {min(lengths)}-{max(lengths)} amino acids")
print(f"✅ READY FOR FEATURE EXTRACTION AND ML ANALYSIS")

# Save balanced protein list for next notebook
balanced_dataset = {
    "total_proteins": len(balanced_proteins),
    "ec_classes": balanced_classes,
    "proteins_by_ec": {ec: ec_distribution[ec] for ec in balanced_classes},
    "protein_ids": balanced_proteins,
    "excluded_ec5": ec_distribution["5"]  # Keep track of excluded proteins
}

# Save to a separate file for easy loading
import json
balanced_file = registry.registry_file.parent / "balanced_dataset.json"
with open(balanced_file, 'w') as f:
    json.dump(balanced_dataset, f, indent=2)

print(f"\n--- FILES CREATED ---")
print(f"Complete registry: {registry.registry_file}")
print(f"Balanced dataset: {balanced_file}")
print(f"PDB structures: {registry.data_source.cache_dir}")

print(f"\n--- READY FOR NEXT PHASE ---")
print(f"🎯 DATA COLLECTION: COMPLETE ✅")
print(f"📊 Dataset: 33 proteins across 4 EC classes")
print(f"🔬 Next step: Feature extraction (02_feature_extraction.ipynb)")
print(f"🤖 ML training: Will use 4-class classification (EC 1,2,3,4)")

=== FINAL DATASET STRATEGY ===
EC 1 (Oxidoreductases):  6 proteins
EC 2 (Transferases   ):  8 proteins
EC 3 (Hydrolases     ): 11 proteins
EC 4 (Lyases         ):  8 proteins

Balanced Dataset Summary:
  4 EC classes: ['1', '2', '3', '4']
  Total proteins: 33
  Average per class: 8.2
  Range: 6-11 proteins per class

📊 FINAL DATASET ASSESSMENT:
✅ EXCELLENT: 33 proteins across 4 balanced EC classes
✅ High quality: Average resolution 1.85Å
✅ Good size distribution: 58-297 amino acids
✅ READY FOR FEATURE EXTRACTION AND ML ANALYSIS

--- FILES CREATED ---
Complete registry: ..\data\processed\protein_registry.json
Balanced dataset: ..\data\processed\balanced_dataset.json
PDB structures: ..\data\raw

--- READY FOR NEXT PHASE ---
🎯 DATA COLLECTION: COMPLETE ✅
📊 Dataset: 33 proteins across 4 EC classes
🔬 Next step: Feature extraction (02_feature_extraction.ipynb)
🤖 ML training: Will use 4-class classification (EC 1,2,3,4)
